In [1]:
include("preamble.jl")

/Users/a1627293/Documents/SFFMProject/DFQExamples.jl/notebooks


  Activating environment at `~/Documents/SFFMProject/DFQExamples.jl/Project.toml`


   Resolving package versions...


  No Changes to `~/Documents/SFFMProject/DFQExamples.jl/Project.toml`
  No Changes to `~/Documents/SFFMProject/DFQExamples.jl/Manifest.toml`


Precompiling project...


...

LoadError: LoadError: 

UndefVarError: Generator not defined
Stacktrace:
  [1] top-level scope
    @ ~/Documents/SFFMProject/DiscretisedFluidQueues.jl/src/1_SFM.jl:

112
  [2] include(mod::

Module, _path::String)
    @ Base ./Base.jl:386
  [3] include(x::String)
    @ DiscretisedFluidQueues ~/Documents/SFFMProject/DiscretisedFluidQueues.jl/src/DiscretisedFluidQueues.jl:1
  [4] top-level scope
    @ ~/Documents/SFFMProject/DiscretisedFluidQueues.jl/src/DiscretisedFluidQueues.jl:25
  [5] include
    @ ./Base.jl:386 [inlined]
  [6] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt64}}, source::String)
    @ Base ./loading.jl:1235
  [7] top-level scope
    @ none:1
  [8] eval
    @ ./boot.jl:360 [inlined]
  [9] eval(x::Expr)
    @ Base.MainInclude ./client.jl:446
 [10] top-level scope
    @ none:1
in expression starting at /Users/a1627293/Documents/SFFMProject/DiscretisedFluidQueues.jl/src/1_SFM.jl:112
in expression starting at /Users/a1627293/Documents/SFFMProject/DiscretisedFluidQueues.jl/src/DiscretisedFluidQueues.jl:1


LoadError: LoadError: Failed to precompile DiscretisedFluidQueues [8d825d2a-a153-45be-9001-3598b904770d] to /Users/a1627293/.julia/compiled/v1.6/DiscretisedFluidQueues/jl_Vsloik.
in expression starting at /Users/a1627293/Documents/SFFMProject/DFQExamples.jl/notebooks/preamble.jl:8

In [2]:
T = Matrix([0.0]')
c = [1.0]
model = FluidQueue(T,c)

nodes = collect(0.0:0.5:5);

UndefVarError: UndefVarError: FluidQueue not defined

In [3]:

for t in [1.0; 1.25]
    for plt_type in (cdf,pdf)
        p = plot(layout=(3,4))
        linetypes = [:solid,:solid,:solid]
        c_order = 0
        for order in 1:2:7
            c_order += 1
            c_mesh = 0
            for mtype in (DGMesh, FVMesh, FRAPMesh)
                c_mesh += 1
                mesh = mtype(nodes,order)

                dq = DiscretisedFluidQueue(model,mesh)
                B = build_full_generator(dq)

                d0 = interior_point_mass(eps(),1,dq)
                dt = integrate_time(d0,B,t,RungeKutta4(0.01))

                rec = plt_type(dt)
                
                std_plot(args...; kwargs...) = plot!(
                    p.layout.grid[c_mesh,c_order],
                    x->rec(x,1),
                    nodes[1]-0.1, 3.6;
                    ylim=((plt_type==pdf) ? (-3,9) : (-0.1,1.2)),
                    linestyle=linetypes[c_mesh],
                    xticks=false,
                    yticks=false,
                    grid=false,
                    kwargs...,
                )
                if c_order == 1
                    std_plot(;label=false,ylabel=uppercase(string(plt_type)),legend=false)
                    yticks!(p.layout.grid[c_mesh,c_order], ((plt_type==pdf) ? (-3:3:9) : (-0.2:0.4:1.2)))
                elseif c_order == 4
                    std_plot(;label=string(mtype),legend=:outerright)
                else
                    std_plot(;label=false,legend=false)
                end
                if c_mesh == 1 
                    plot!(p.layout.grid[c_mesh,c_order]; title="Order "*string(order))
                elseif c_mesh == 3
                    xticks!(p.layout.grid[c_mesh,c_order], 0:1:3.6)
                    plot!(p.layout.grid[c_mesh,c_order]; xlabel="x")
                end
                std_plot2(args... ; kwargs...) = if plt_type==cdf
                    plot!(
                        p.layout.grid[c_mesh,c_order],
                        [0.0;t+eps();t+eps();3.6],
                        [0.0;0.0;1.0;1.0],
                        args...;
                        linestyle=:dot,
                        kwargs...,
                    )
                elseif plt_type==pdf
                    plot!(
                        p.layout.grid[c_mesh,c_order],
                        [0.0;t;t+eps();t+2*eps();3.6],
                        [0.0;0.0;100;0.0;0.0],
                        args...;
                        subplot=c_order,
                        linestyle=:dot,
                        kwargs...
                    )
                end
                if c_order==4
                    std_plot2(;label="truth",legend=:outerright)
                else
                    std_plot2(;label=false)
                end
            end
        end
        display(p)
    end
end

UndefVarError: UndefVarError: cdf not defined